# Approximation Algorithms

[Arxiv Paper](https://arxiv.org/pdf/2205.15874.pdf)

In [ ]:
from typing import Dict, List, Iterable
import numpy as np
import cvxpy as cp
from math import exp, log
import utils

### Theorem 5.1 ($\ell \le 0$)

In [ ]:
def r4(x):
    return round(x, 4)

def thm_ell_neg(betas: Iterable[float], csm: bool) -> List[float]:
    keys = ['f_OPT', 'f_z_and_OPT', 'f_z_or_OPT', 'L_OPT']
    linear_combs = []
    labels = []
    def add_linear_comb(d: Dict, label: str):
        val = [0] * len(keys)
        for k, v in d.items():
            val[keys.index(k)] = v
        linear_combs.append(val)
        labels.append(label)
    add_linear_comb({}, "nothing") # (0, 0, 0, 0)
    add_linear_comb({'f_z_and_OPT': 0.5, 'f_z_or_OPT': 0.5, 'L_OPT': 1}, "local_search_1") # (0, 0.5, 0.5, 1.0)
    add_linear_comb({'f_z_and_OPT': 1.0, 'L_OPT': 1}, "local_search_2")
    for t_f in np.linspace(0, 2, 41):
        for t_s in np.linspace(0, 2, 41):
            if t_s <= t_f:
                if csm:
                    if t_f > 1 + utils.EPS:
                        continue
                add_linear_comb(
                    {
                        'f_OPT': exp(t_s-t_f) + t_f * exp(t_s-t_f) - t_s * exp(t_s-t_f) - exp(-t_f),
                        'f_z_and_OPT': -exp(t_s-t_f) + exp(-t_f),
                        'f_z_or_OPT': -exp(t_s-t_f) - t_f * exp(t_s-t_f) + t_s * exp(t_s-t_f) + exp(-t_f) + t_f * exp(-t_f),
                        'L_OPT': t_f,
                    }, 
                    f'measured_{r4(t_s)}_{r4(t_f)}'
                )
    
    weights = cp.Variable(len(linear_combs), nonneg=True)
    sum_vars = []
    for j in range(len(keys)):
        sum_var = 0
        for i in range(len(linear_combs)):
            sum_var += linear_combs[i][j] * weights[i]
        sum_vars.append(sum_var)
    
    results = []
    for beta in betas:
        constraints = [sum(weights) == 1, sum_vars[1] >= 0, sum_vars[2] >= 0, sum_vars[3] <= beta]
        prob = cp.Problem(cp.Maximize(sum_vars[0]), constraints)
        result = prob.solve()
        print("beta =", beta, "result =", result)
        print("contributions:")
        for i in range(len(linear_combs)):
            if weights[i].value > utils.EPS:
                print(labels[i], weights[i].value)
        print()
        results.append(result)
    
    return results

In [ ]:
ell_neg_usm = thm_ell_neg(np.linspace(0, 1.4, num=140+1), csm=False)
ell_neg_usm

beta = 0.0 result = 1.6935812991431703e-10
contributions
nothing 0.49999999978874404
measured_0.0_0.0 0.49999999978874404

beta = 0.01 result = 0.009512294693261026
contributions
nothing 0.40000000140706166
measured_0.0_0.0 0.40000000140706166
measured_0.0_0.05 0.19999999398080245

beta = 0.02 result = 0.019024588505687858
contributions
nothing 0.3000000000619218
measured_0.0_0.0 0.3000000000619218
measured_0.0_0.05 0.39999999978119133

beta = 0.03 result = 0.02853688342677501
contributions
nothing 0.20000000358980466
measured_0.0_0.0 0.20000000358980466
measured_0.0_0.05 0.599999987344917

beta = 0.04 result = 0.038049177237314394
contributions
nothing 0.1000000000554776
measured_0.0_0.0 0.1000000000554776
measured_0.0_0.05 0.7999999986761047

beta = 0.05 result = 0.04756147126436101
contributions
measured_0.0_0.05 0.9999999973252929

beta = 0.06 result = 0.056145925350583784
contributions
measured_0.0_0.05 0.8000000001311447
measured_0.0_0.1 0.1999999997382739

beta = 0.07 result = 0

[1.6935812991431703e-10,
 0.009512294693261026,
 0.019024588505687858,
 0.02853688342677501,
 0.038049177237314394,
 0.04756147126436101,
 0.056145925350583784,
 0.06473037976343975,
 0.07331483386810714,
 0.08189928769484933,
 0.0904837419838457,
 0.09820823273992606,
 0.10593272368803336,
 0.11365721465105828,
 0.12138170558141603,
 0.12910619648199806,
 0.1360341873083371,
 0.1429621781339049,
 0.14989016906481892,
 0.15681815980361258,
 0.1637461506586881,
 0.16993695966927913,
 0.1761277687004397,
 0.1823185777101439,
 0.18850938685300678,
 0.19470019585273843,
 0.20020924987436964,
 0.20571830395563243,
 0.2112273580348474,
 0.2167364121618734,
 0.22224546622123179,
 0.2271245392461987,
 0.2320036124748374,
 0.23688268532676474,
 0.24176175845216308,
 0.24664083140494636,
 0.2509382689011808,
 0.2552357062459843,
 0.2595331436336315,
 0.2638305811632357,
 0.2681280184367834,
 0.27188894837949296,
 0.2756498783924005,
 0.2794108083234606,
 0.2831717382941946,
 0.28693266823204266,

## Section 5 Table 1 ($\ell \le 0$)

In [ ]:
table = []
for i in range(5, 141):
    if i%10 == 0:
        table.append([round(0.01*i, 2), round(ell_neg_usm[i], 4)])
for i in range(7):
    print(table[i][0], '&', table[i][1], '&', table[i+7][0], '&', table[i+7][1], '\\\\')

0.1 & 0.0905 & 0.8 & 0.363 \\
0.2 & 0.1637 & 0.9 & 0.3757 \\
0.3 & 0.2222 & 1.0 & 0.3856 \\
0.4 & 0.2681 & 1.1 & 0.3925 \\
0.5 & 0.3033 & 1.2 & 0.3967 \\
0.6 & 0.3293 & 1.3 & 0.3982 \\
0.7 & 0.3478 & 1.4 & 0.3982 \\


In [ ]:
def gen_lu_et_al():
    for beta in np.linspace(0, 1.4, 15):
        print(round(beta, 4), round(beta*exp(-beta), 4))

gen_lu_et_al()

0.0 0.0
0.1 0.0905
0.2 0.1637
0.3 0.2222
0.4 0.2681
0.5 0.3033
0.6 0.3293
0.7 0.3476
0.8 0.3595
0.9 0.3659
1.0 0.3679
1.1 0.3662
1.2 0.3614
1.3 0.3543
1.4 0.3452


In [ ]:
ell_neg_csm = thm_ell_neg(np.linspace(0, 1.0, num=100+1), csm=True)
ell_neg_csm

beta = 0.0 result = 1.0180590221663363e-11
contributions:
nothing 0.4999999999776724
measured_0.0_0.0 0.4999999999776724

beta = 0.01 result = 0.009512294334414428
contributions:
nothing 0.40000000056960633
measured_0.0_0.0 0.40000000056960633
measured_0.0_0.05 0.19999999688429823

beta = 0.02 result = 0.019024588585388715
contributions:
nothing 0.30000000034511337
measured_0.0_0.0 0.30000000034511337
measured_0.0_0.05 0.3999999978526211

beta = 0.03 result = 0.028536882746117678
contributions:
nothing 0.20000000001378254
measured_0.0_0.0 0.20000000001378254
measured_0.0_0.05 0.5999999999135974

beta = 0.04 result = 0.03804917704862066
contributions:
nothing 0.10000000027141356
measured_0.0_0.0 0.10000000027141356
measured_0.0_0.05 0.7999999987231936

beta = 0.05 result = 0.047561471230423084
contributions:
measured_0.0_0.05 0.9999999996722824

beta = 0.06 result = 0.05614592539364061
contributions:
measured_0.0_0.05 0.7999999986509418
measured_0.0_0.1 0.1999999990858268

beta = 0.07 r

[1.0180590221663363e-11,
 0.009512294334414428,
 0.019024588585388715,
 0.028536882746117678,
 0.03804917704862066,
 0.047561471230423084,
 0.05614592539364061,
 0.06473037947879183,
 0.07331483359597894,
 0.08189928769216523,
 0.09048374180975695,
 0.09820823278175068,
 0.10593272371064946,
 0.11365721460686672,
 0.12138170553611831,
 0.12910619651169156,
 0.13603418729582684,
 0.1429621781285223,
 0.1498901690474302,
 0.15681815984555666,
 0.16374615061955003,
 0.16993695971503958,
 0.17612776869664612,
 0.18231857771072768,
 0.18850938682347984,
 0.1947001957951922,
 0.2002092498606682,
 0.20571830395172191,
 0.21122735808444296,
 0.21673641214550335,
 0.22224546620854121,
 0.22712453929288692,
 0.2320036122843377,
 0.2368826853474081,
 0.24176175837551456,
 0.24664083142179882,
 0.2509382688080885,
 0.2552357062113337,
 0.25953314361823365,
 0.2638305810231554,
 0.26812801841833855,
 0.27188894838711547,
 0.2756498783416971,
 0.2794108083375568,
 0.2831717382808193,
 0.286932668239

In [ ]:
# check that USM and CSM essentially the same
max_dif = 0
for i in range(101):
    max_dif = max(max_dif, abs(ell_neg_usm[i] - ell_neg_csm[i]))
print(max_dif)

6.80657332580159e-10


## Theorem 6.5 ($\ell \ge 0$), Section 6 Table 3


In [ ]:
ell_neg_usm_pairs = list(zip(np.linspace(0, 1.4, 141), ell_neg_usm))
ell_neg_usm_pairs

[(0.0, 1.6935812991431703e-10),
 (0.01, 0.009512294693261026),
 (0.02, 0.019024588505687858),
 (0.03, 0.02853688342677501),
 (0.04, 0.038049177237314394),
 (0.05, 0.04756147126436101),
 (0.06, 0.056145925350583784),
 (0.07, 0.06473037976343975),
 (0.08, 0.07331483386810714),
 (0.09, 0.08189928769484933),
 (0.1, 0.0904837419838457),
 (0.11, 0.09820823273992606),
 (0.12, 0.10593272368803336),
 (0.13, 0.11365721465105828),
 (0.14, 0.12138170558141603),
 (0.15, 0.12910619648199806),
 (0.16, 0.1360341873083371),
 (0.17, 0.1429621781339049),
 (0.18, 0.14989016906481892),
 (0.19, 0.15681815980361258),
 (0.2, 0.1637461506586881),
 (0.21, 0.16993695966927913),
 (0.22, 0.1761277687004397),
 (0.23, 0.1823185777101439),
 (0.24, 0.18850938685300678),
 (0.25, 0.19470019585273843),
 (0.26, 0.20020924987436964),
 (0.27, 0.20571830395563243),
 (0.28, 0.2112273580348474),
 (0.29, 0.2167364121618734),
 (0.3, 0.22224546622123179),
 (0.31, 0.2271245392461987),
 (0.32, 0.2320036124748374),
 (0.33, 0.2368826

In [ ]:
def thm_ell_pos_usm(betas: Iterable[float]) -> List[float]:
    linear_combs = []
    labels = []
    
    def add_linear_comb(l: list, label: str):
        assert len(l) == 3
        linear_combs.append(l)
        labels.append(label)
    
    for beta, alpha in ell_neg_usm_pairs:
        add_linear_comb([alpha, beta, 1 - beta], f"ell_neg_usm_{r4(beta)}_{alpha}")
    
    for r in np.linspace(1, 2, 1001):
        add_linear_comb([2 / (r+1/r)**2, 2 / (r+1/r)**2, (r**2)/(r+1/r)**2], f"double_greedy_{r}")
    
    weights = cp.Variable(len(linear_combs), nonneg=True)
    sum_vars = []
    for i in range(3):
        sum_var = 0
        for j, linear_comb in enumerate(linear_combs):
            sum_var += weights[j] * linear_comb[i]
        sum_vars.append(sum_var)

    results = []
    for beta in betas:
        constraints = [sum(weights) == 1, sum_vars[1] + sum_vars[2] >= beta, sum_vars[2] >= 0]
        prob = cp.Problem(cp.Maximize(sum_vars[0]), constraints)
        result = prob.solve()
        print("beta =", beta, "result =", result)
        print("contributions")
        for i in range(len(linear_combs)):
            if weights[i].value > utils.EPS:
                print(labels[i], weights[i].value)
        print()
        results.append(result)
    
    return results    

results_ell_pos_usm = thm_ell_pos_usm(np.linspace(0.85, 1, 4))
for r in results_ell_pos_usm:
    print(round(r, 4))

beta = 0.85 result = 0.47489513944380174
contributions
ell_neg_usm_1.29_0.39820101596527774 0.00476548901743604
ell_neg_usm_1.3_0.39820101594947527 0.0045513613190747365
ell_neg_usm_1.31_0.39820101594853197 0.0045323219900557
ell_neg_usm_1.32_0.39820101594908114 0.004532181603033886
ell_neg_usm_1.33_0.3982010159629967 0.004706561188109847
ell_neg_usm_1.34_0.398201015959116 0.004647555953825672
ell_neg_usm_1.35_0.3982010159504843 0.004527523847461855
ell_neg_usm_1.36_0.39820101594726776 0.0044785062903116325
ell_neg_usm_1.37_0.3982010159474121 0.004472230710883152
ell_neg_usm_1.38_0.398201015953414 0.0045409081878133965
ell_neg_usm_1.39_0.39820101594714974 0.004452103988156331
ell_neg_usm_1.4_0.3982010159473649 0.004446152410808437
double_greedy_1.229 0.9453466333399626

beta = 0.9 result = 0.44933043161037867
contributions
ell_neg_usm_1.29_0.39820101596527774 0.03202762567412679
ell_neg_usm_1.3_0.39820101594947527 0.03137351394912482
ell_neg_usm_1.31_0.39820101594853197 0.0311874298250

## Theorem 7.1 (RegularizedCSM $\ell \ge 0$)

In [ ]:
def thm_ell_pos(betas: Iterable[float]) -> List[float]:
    keys = ['f_OPT', 'f_z_and_OPT', 'f_z_or_OPT', 'L_OPT_backslash_z', 'L_OPT_and_z']
    linear_combs = []
    labels = []
    def add_linear_comb(d: Dict, label: str):
        val = [0] * len(keys)
        for k, v in d.items():
            val[keys.index(k)] = v
        linear_combs.append(val)
        labels.append(label)
    # (0, 0, 0, 1, 1)
    add_linear_comb({'L_OPT_backslash_z': 1, 'L_OPT_and_z': 1}, "trivial")
    # (0, 0.5, 0.5, 0.5, 1.0)
    add_linear_comb({'f_z_and_OPT': 0.5, 'f_z_or_OPT': 0.5, 'L_OPT_backslash_z': 0.5, 'L_OPT_and_z': 1.0}, "local_search_1")
    # (0, 1, 0, 0, 1)
    add_linear_comb({'f_z_and_OPT': 1.0, 'L_OPT_and_z': 1.0}, "local_search_2")
    # for t_f in np.linspace(0, 1, 41):
    t_f = 1
    for t_s in np.linspace(0, 1, 101):
        if t_s <= t_f:
            add_linear_comb(
                {
                    'f_OPT': exp(t_s-t_f) + t_f * exp(t_s-t_f) - t_s * exp(t_s-t_f) - exp(-t_f),
                    'f_z_and_OPT': -exp(t_s-t_f) + exp(-t_f),
                    'f_z_or_OPT': -exp(t_s-t_f) - t_f * exp(t_s-t_f) + t_s * exp(t_s-t_f) + exp(-t_f) + t_f * exp(-t_f),
                    'L_OPT_backslash_z': 1-exp(-t_f),
                    'L_OPT_and_z': 1-exp(-t_f+t_s),
                }, 
                f'measured_{t_s}_{t_f}'
            )
    
    weights = cp.Variable(len(linear_combs), nonneg=True)
    sum_vars = []
    for j in range(len(keys)):
        sum_var = 0
        for i in range(len(linear_combs)):
            sum_var += linear_combs[i][j] * weights[i]
        sum_vars.append(sum_var)
    
    results = []
    for beta in betas:
        constraints = [sum(weights) == 1, sum_vars[1] >= 0, sum_vars[2] >= 0, sum_vars[3] >= beta, sum_vars[4] >= beta]
        prob = cp.Problem(cp.Maximize(sum_vars[0]), constraints)
        result = prob.solve()
        print("beta =", beta, "result =", result)
        print("contributions")
        for i in range(len(linear_combs)):
            if weights[i].value > utils.EPS:
                print(labels[i], weights[i].value)
        print()
        results.append(result)
    
    return results

In [ ]:
thm_ell_pos(np.linspace(0, 1, 101))

beta = 0.0 result = 0.3856708315441379
contributions
local_search_1 0.20409835299636284
local_search_2 0.024938055939882083
measured_0.37_1 0.77096358544296

beta = 0.01 result = 0.3856708315441489
contributions
local_search_1 0.20409835299635198
local_search_2 0.024938055939880143
measured_0.37_1 0.7709635854381856

beta = 0.02 result = 0.38567083154415865
contributions
local_search_1 0.20409835299634402
local_search_2 0.024938055939877538
measured_0.37_1 0.7709635854351609

beta = 0.03 result = 0.38567083154416926
contributions
local_search_1 0.2040983529963385
local_search_2 0.0249380559398733
measured_0.37_1 0.7709635854339283

beta = 0.04 result = 0.3856708315441797
contributions
local_search_1 0.20409835299633639
local_search_2 0.024938055939868742
measured_0.37_1 0.7709635854345115

beta = 0.05 result = 0.38567083154419013
contributions
local_search_1 0.20409835299633675
local_search_2 0.024938055939862504
measured_0.37_1 0.7709635854369458

beta = 0.06 result = 0.38567083154420

[0.3856708315441379,
 0.3856708315441489,
 0.38567083154415865,
 0.38567083154416926,
 0.3856708315441797,
 0.38567083154419013,
 0.38567083154420034,
 0.3856708315442106,
 0.3856708315442204,
 0.3856708315442307,
 0.3856708315442412,
 0.385670831544251,
 0.385670831544261,
 0.3856708315442703,
 0.3856708315442803,
 0.3856708315442895,
 0.38567083154429865,
 0.3856708315443073,
 0.3856708315443163,
 0.38567083154432485,
 0.3856708315443332,
 0.3856708315443409,
 0.38567083154434983,
 0.38567083154436205,
 0.3856708315443746,
 0.38567083154438725,
 0.3856708315443989,
 0.3856708315444105,
 0.38567083154442167,
 0.385670831544432,
 0.3856708315444423,
 0.38567083154445164,
 0.3856708315444605,
 0.3856708315444695,
 0.3856708315444785,
 0.3856708315444866,
 0.38567083154449305,
 0.3856708315444993,
 0.3856708315445057,
 0.3856708315445123,
 0.3856708315445189,
 0.38567083154452536,
 0.3856708315445317,
 0.3856708315445377,
 0.3856708315445455,
 0.3856708315445549,
 0.3856708315445633,
 0.

## Theorem 8.3, Conference Theorem 5 ($\ell$ Unconstrained)

In [ ]:
def thm_ell_pos(betas: Iterable[float]) -> List[float]:
    keys = ['f_OPT', 'f_z_and_OPT', 'f_z_or_OPT', 'L_OPT_backslash_z', 'L_OPT_and_z', 'L_OPT_backslash_z_neg', 'L_OPT_and_z_neg']
    linear_combs = []
    labels = []
    def add_linear_comb(d: Dict, label: str):
        val = [0] * len(keys)
        for k, v in d.items():
            val[keys.index(k)] = v
        linear_combs.append(val)
        labels.append(label)
    # (0, 0, 0, 1, 1)
    add_linear_comb({'L_OPT_backslash_z': 1, 'L_OPT_and_z': 1}, "trivial")
    # (0, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0)
    add_linear_comb({'f_z_and_OPT': 0.5, 'f_z_or_OPT': 0.5, 'L_OPT_backslash_z': 0.5, 'L_OPT_and_z': 1.0, 'L_OPT_backslash_z_neg': 1.0, 'L_OPT_and_z_neg': 1.0}, "local_search_1")
    # (0, 1, 0, 0, 1, 0.0, 1.0)
    add_linear_comb({'f_z_and_OPT': 1.0, 'L_OPT_and_z': 1.0, 'L_OPT_and_z_neg': 1.0}, "local_search_2")
    # for t_f in np.linspace(0, 1, 41):
    for (t_s, t_f) in [(0.205, 0.955)]:
        add_linear_comb(
            {
                'f_OPT': exp(t_s-t_f) + t_f * exp(t_s-t_f) - t_s * exp(t_s-t_f) - exp(-t_f),
                'f_z_and_OPT': -exp(t_s-t_f) + exp(-t_f),
                'f_z_or_OPT': -exp(t_s-t_f) - t_f * exp(t_s-t_f) + t_s * exp(t_s-t_f) + exp(-t_f) + t_f * exp(-t_f),
                'L_OPT_backslash_z': 1-exp(-t_f),
                'L_OPT_and_z': 1-exp(-t_f+t_s),
                'L_OPT_backslash_z_neg': t_f,
                'L_OPT_and_z_neg': t_f-t_s,
            }, 
            f'measured_{t_s}_{t_f}'
        )
    
    weights = cp.Variable(len(linear_combs), nonneg=True)
    sum_vars = []
    for j in range(len(keys)):
        sum_var = 0
        for i in range(len(linear_combs)):
            sum_var += linear_combs[i][j] * weights[i]
        sum_vars.append(sum_var)
    
    results = []
    for beta in betas:
        constraints = [sum(weights) == 1, sum_vars[1] >= 0, sum_vars[2] >= 0, sum_vars[3] >= beta, sum_vars[4] >= beta, sum_vars[5] <= beta, sum_vars[6] <= beta]
        prob = cp.Problem(cp.Maximize(sum_vars[0]), constraints)
        result = prob.solve()
        print("beta =", beta, "result =", result)
        print("contributions")
        for i in range(len(linear_combs)):
            if weights[i].value > utils.EPS:
                print(labels[i], weights[i].value)
        print()
        results.append(result)
    
    return results

thm_ell_pos([0.7])

beta = 0.7 result = 0.2802298156858885
contributions
trivial 0.2628936800139511
local_search_1 0.09429218432600248
local_search_2 0.008565114011325627
measured_0.205_0.955 0.6342490216485358



[0.2802298156858885]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6cbc651c-00ab-4e5c-98d9-adde0f700e67' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>